In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns # 시각화 라이브러리


### 목표
- 타이타닉 데이터를 학습해서 생존자 및 사망자를 예측해보자
- 머신러닝 전체 과정 진행

### 1. 문제정의
- 생존자/사망자 예측
- 머신러닝 과정 전체를 체험해보는 예제

### 2. 데이터 수집
- Kaggle 사이트로부터 다운로드

In [13]:
# 타이타닉 train데이터와 test데이터를 변수 train, test에 저장하시오
# 단 인덱스는 PassengerId 사용

In [14]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [15]:
train.shape

(6999, 12)

In [16]:
test.shape

(4000, 11)

### 3. 데이터 전처리 및 데이터탐색

In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6999 entries, 0 to 6998
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   6999 non-null   int64  
 1   Warehouse_block      6999 non-null   object 
 2   Mode_of_Shipment     6999 non-null   object 
 3   Customer_care_calls  5423 non-null   float64
 4   Customer_rating      6999 non-null   int64  
 5   Cost_of_the_Product  6999 non-null   int64  
 6   Prior_purchases      6049 non-null   float64
 7   Product_importance   6999 non-null   object 
 8   Gender               6999 non-null   object 
 9   Discount_offered     3468 non-null   float64
 10  Weight_in_gms        6999 non-null   object 
 11  Reached.on.Time_Y.N  6999 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 656.3+ KB


### Age 채우기
- 단순 기술통계치로 채우지않고 다른 컬럼과의 상관관계를 통해 결측치를 채워보자.
- Age와 다른 컬럼간의 상관관계를 알아보자.

In [18]:
# corr()      
# 상관계수(correlation) : 변수간의 관계의 정도를 수치로 요약해주는 지표
# -1 ~ 1 사이의 값으로 표현
# -1에 가까울수록 음의 상관관계 -> 반비례관계
# 1에 가까울수록 양의 상관관계 -> 비례관계

#결국 가장높은 상관관계를 알아보려면 절대값으로 !! 


train.corr()

,ID,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Reached.on.Time_Y.N
ID,1.000000,-0.007719,0.007690,0.009738,-0.002985,0.002196,-0.014909
Customer_care_calls,-0.007719,1.000000,0.013420,0.047041,0.173709,-0.158465,-0.070266
Customer_rating,0.007690,0.013420,1.000000,-0.001961,0.008217,-0.016456,0.011275
Cost_of_the_Product,0.009738,0.047041,-0.001961,1.000000,0.028947,-0.139349,-0.012811
Prior_purchases,-0.002985,0.173709,0.008217,0.028947,1.000000,-0.081868,-0.065160
Discount_offered,0.002196,-0.158465,-0.016456,-0.139349,-0.081868,1.000000,0.397393
Reached.on.Time_Y.N,-0.014909,-0.070266,0.011275,-0.012811,-0.065160,0.397393,1.000000


- Pclass가 가장 높은 상관관계를 갖는다.
- 생존에 관련이 많은 것 같은 성별을 함께 활용

In [19]:
pt1 = train.pivot_table( values='Age',
                 index = ['Pclass', 'Sex'],
                 aggfunc = 'mean')
pt1

KeyError: 'Age'

In [ ]:
pt1.loc[2, 'female']

In [ ]:
pt1.loc[3,'male'][0]

In [ ]:
def fill_age(row):
    # 만약 나이가 결측치라면 피봇테이블에서 값을 가져와서 채우겠다.
    if np.isnan(row['Age']):
        return pt1.loc[row['Pclass'], row['Sex']][0]
    # 만약 나이가 결측치가 아니라면 원래 나이값을 사용
    else:
        return row['Age']

In [ ]:
train['Age'] = train.apply(fill_age, axis = 1)

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
test['Age'] = test.apply(fill_age, axis = 1)


In [ ]:
test.info()

### Embarked 채우기

In [ ]:
train['Embarked'].value_counts()

In [ ]:
test['Embarked'].value_counts()

In [ ]:
# fillna('값') : 결측치에 값을 채움

In [ ]:
train['Embarked'] = train['Embarked'].fillna('S')

### Fare 채우기

In [ ]:
# 1. 상관관계를 파악하여 가장 관계가 깊은 컬럼찾기
train.corr() #Pclass



In [ ]:
# 2. 관계가 깊은컬럼과, 성별을 같이 묶어 피봇테이블 생성
pt2 = train.pivot_table( values='Fare',
                 index = ['Pclass', 'Sex'],
                 aggfunc = 'mean')
pt2




In [ ]:
#결측치 1개 -> 직접 확인해보자 !
test[test['Fare'].isnull()]

In [ ]:
pt2.loc[3,'female']
# 시리즈 정보가 섞여있음 . 데이터를 넣어줄땐 숫자값만 넣어주기 위해 아래에서 [0]넣어서 숫자값만 들어가도록해줘야함.
#안그럼 이 정보들이 시리즈 형태로 다 들어가게되면 나중에 핫원 인코딩이 안됌.

In [ ]:
# 3. apply 함수를 활용할 사용자 정의함수 구현( fill_fare()로 만들기 )  - apply함수 --> for문 이용안하고 하나하나씩 넣어주는 함수
def fill_fare(row):
    # 만약 나이가 결측치라면 피봇테이블에서 값을 가져와서 채우겠다.
    if np.isnan(row['Fare']):
        return pt2.loc[row['Pclass'], row['Sex']][0]
    # 만약 나이가 결측치가 아니라면 원래 나이값을 사용
    else:
        return row['Fare']
# 4. apply 함수 사용하여 결측치 채우기. 
test['Fare']= test.apply(fill_fare, axis = 1)

In [ ]:
test.info()

In [ ]:
#이게 결측치 값을 채우는 과정임

### Cabin 결측치 채우기

In [ ]:
train['Cabin']

In [ ]:
train['Cabin'].unique()

- Cabin의 범주가 너무 많으니 종류를 줄여보자!(모델이 복잡해지는것 (과대적합)을 방지)
- 결측치를 하나의 데이터로 활용해볼수도있음

In [ ]:
train['Cabin'].str[0]
#pandas에서 제공하는 문자열 각 하나하나에 접근하는 str

In [ ]:
train['Cabin']

In [ ]:
train.info()

In [ ]:
#결측치 채우기--> 기존값에 없는 알파벳으로 채워줘볼거임
train['Cabin'].fillna('M', inplace= True)

In [ ]:
train['Cabin']

In [ ]:
#train에 저렇게 했으면 test에도 동일하게 해줘야함( train에 해줬던 전처리를 동일하게 해주기)
test['Cabin']=test['Cabin'].str[0]

In [ ]:
test['Cabin'].fillna('M', inplace = True)

In [ ]:
test.info()

- 분석전에 전처리 해줬음


### 4. 탐색적 데이터 분석(EDA)
- train 데이터를 이용해 탐색

#### 범주형 데이터 시각화
- 빈도기반에 bar chart 많이 활용

### Cabin 컬럼 시각화

In [ ]:
sns.countplot(data=train,
                 x='Cabin',
                hue='Survived')
#씨본을 이용해서 카운트플롯을 그릴거야 이때 사용할 데이터는 train데이터이고 train데이터에있는 cabin컬럼을 x축에 넣어

- M에서 상대적으로 많은 사람들이 사망했다. 모델에게 학습을 시켜도 의미가 있을 수 있겠다 


 ##### Pclass 컬럼 시각화

In [ ]:
sns.countplot(data=train,
             y='Pclass',
              hue='Survived'  #생존자와 사망자 나누어서 쪼개어보기
             )

- 1등급에 탑승하면 생존 확률이 조금 있다.
- 3등급에 탑승하면 사망 확률이 높다.
- Pclass를 모델에게 사용해도 괜찮을 것 같다!

### Pclass와 Cabin 시각화

In [ ]:
sns.countplot(data=train,
             x='Cabin',
             hue='Pclass')

- A,B,C 에는 1등급 사람들이 탑승했다.
- M에는 3등급 사람들이 많이 탑승했다 -> 3등급 사람이 많이 사망해서 정보가 손실된걸까?

#### Embarked, Sex 시각화

In [ ]:
sns.countplot(data=train,
             x='Embarked',
             hue='Survived')

In [ ]:
sns.countplot(data=train,
             x='Embarked',
             hue='Pclass')

In [ ]:
sns.countplot(data=train,
             x='Sex',
             hue='Survived')

### 수치형 데이터

In [ ]:
#데이터 타입 int로 변경
train['Age'] = train['Age'].astype('int')

In [ ]:
test['Age']=test['Age'].astype('int')

In [ ]:
test['Age']

In [ ]:
train['Age']

In [ ]:
plt.hist(train['Age'])
plt.show()

In [ ]:
plt.hist(train['Age'],bins=3)
plt.show()

In [ ]:
# 밀도그래프 !
plt.figure(figsize=(15,5))
sns.violinplot(data=train,
              y='Age',
              x='Sex',
              hue='Survived',
              split=True)

- 어린아이 중 남자아이는 생존 확률이 높고 여자 아이는 사망확률이 조금더 높다(시대적인 배경?)
- 남성의 경우 2-30대가 좀 더 많이 사망

### Fare 시각화

In [ ]:
# 밀도그래프 !
plt.figure(figsize=(15,5))
sns.violinplot(data=train,
              y='Fare',
              x='Sex',
              hue='Survived',
              split=True)

- 낮은 요금을 지불한 사람들이 상대적으로 많이 사망했다.
- 전체 요금이 0~40달러 사이에 많이 분포되어 있다. 

In [ ]:
train.head(1)

### 특성공학
- SibSp와 Parch를 합쳐서 '가족'이라는 새 컬럼 생성

In [ ]:
train['Family_Size'] = train['SibSp'] + train['Parch'] +1

In [ ]:
test['Family_Size'] = test['SibSp'] + test['Parch']+1

In [ ]:
train.head(1)

In [ ]:
sns.countplot(data=train,
             x='Family_Size',
             hue='Survived')

- 1명은 사망 비율이 높다
- 2~4명은 생존 비율이 조금 더 높다.
- 5명 이상은 사망 비율이 조금 더 높다.
- Family_Size를 그대로 써도 좋지만, 범주형 데이터로 변환해보자!
- Binning(수치형 -> 범주형) - 모델 학습의 단순화 유도

#### 특성공학 
- Family_Size가 1이면 Alone, 2~4 이면 Small, 5명 이상이면 Large범주로 변경!
- cut함수 활용




- 그룹화시키기
- 수치데이터를 범주형데이터로 바꾸는걸 binning이라고함



In [ ]:
train['Family_Size']

In [ ]:
# 구간에따라 나눠주는(잘라주는) cut함수
bins =[0,1,4,20] #구간설정
labels=['Alone','Small','Large'] #구간별 이름 설정

train['Family_Group'] = pd.cut(x=train['Family_Size'],
      bins=bins,
      labels=labels)


In [ ]:
# 구간에따라 나눠주는(잘라주는) cut함수
bins =[0,1,4,20] #구간설정
labels=['Alone','Small','Large'] #구간별 이름 설정

test['Family_Group'] = pd.cut(x=test['Family_Size'],
      bins=bins,
      labels=labels)


In [ ]:
test.head(3)

In [ ]:
test['Family_Group'].unique()

In [ ]:
sns.countplot(data=train,
             x='Family_Group',
             hue='Survived')

In [ ]:
train.head(1)

 #### 특성공학
- text 데이터 다루기(비정형 데이터)
- Name : 중간에 있는 호칭 정보만 뽑아서 정형화 시켜보자 !


In [ ]:
train['Name']

In [ ]:
# 호칭 정보만 뽑아보자 !

# 1. ',' 뒤  --> split 함수로 ,(쉽표)를 가지고 쪼개고 리스트에 담아주는 함수
'Braund, Mr. Owen Harris'

In [ ]:
'Braund, Mr. Owen Harris'.split(',')[1]

In [ ]:
# 2. '.' 앞에 호칭 정보가 있다!
'Braund, Mr. Owen Harris'.split(',')[1].split('.')[0]

In [ ]:
# 3. 공백 제거 !
'Braund, Mr. Owen Harris'.split(',')[1].split('.')[0].strip()


In [ ]:
# 함수 만들기
def split_title(row):
    return row.split(',')[1].split('.')[0].strip()
    

In [ ]:
#이렇게 변경하는 함수를 train 의 name컬럼에 적용시켜주기 
# 내가만든 함수를 전부다 적용시켜주는 함수 apply (for문처럼)
train['Name'].apply(split_title)

In [ ]:
# 호칭 정보가 담긴 title 컬럼 생성 !
train['Title'] = train['Name'].apply(split_title)

In [ ]:
test['Title'] = test['Name'].apply(split_title)

In [ ]:
test.head()

In [ ]:
# 정형화된 Title 종류 확인 !
train['Title'].unique()

In [ ]:
plt.figure(figsize=(20,5))
sns.countplot(data = train,
             x='Title',
             hue='Survived')

- Mr, Mrs, Miss : 성별과 관련이 많은 호칭. 모델에 사용해봐도 괜찮겠다!
- Master : 나이가 어린 남성을 부르는 호칭 
- 나머지 호칭들은 인원수가 적고 종류가 많다 -> 범주를 통합시켜보자 !

In [ ]:
# map 함수 이용( 딕셔너리 자료형에서 키값을 벨류로  하나하나 매핑시켜주는)
title_dic = {
    'Mr' : 'Mr',
    'Mrs' : 'Mrs',
    'Miss' : 'Miss',
    'Master' : 'Master',
    'Mme' : 'Miss',
    'Lady' : 'Miss',
    'Don' : 'Other',
    'Rev' : 'Other',
    'Dr' : 'Other',
    'Ms' : 'Miss',
    'Major' : 'Other',
    'Sir' : 'Other',
    'Mlle' : 'Other',
    'Col' : 'Other',
    'Capt' : 'Other',
    'the Countess' : 'Other',
    'Jonkheer' : 'Other',
    'Dona' : 'Other'
}

In [ ]:
train['Title2'] = train['Title'].map(title_dic)

In [ ]:
train['Title2'].unique() 

In [ ]:
test['Title2'] = test['Title'].map(title_dic)

In [ ]:
test['Title2'].unique()

In [ ]:
test['Title'].unique()

In [ ]:
train.head()

### Ticket 정보는 사용하지 않을거다 !

 ### 사용하지 않을 컬럼 정리해보기  -> 비정형데이터로 이루어진 컬럼  3개 삭제할거야

In [ ]:
train.columns

In [ ]:
train.drop(['Name','Ticket','Title'], axis=1, inplace =True)

In [ ]:
test.drop(['Name','Ticket','Title'], axis=1,inplace=True)

In [ ]:
#---모델링할 데이터 준비 끝-----

### 5. 모델링
- 이대로 모델링 시킬수 없음 --> 문자열 데이터가 있어서 (문자열데이터는 컴퓨터가 알아먹지 못함)
- 컴퓨터가 알아먹을 수 있도록 수치형으로 바꿔주는 인코딩 해줘야함.( label인코딩,one-hot인코딩)

- 인코딩 (문자 형태의 데이터를 숫자 형태의 데이터로 변환)
    - 1. label encoding
    - 2. one-hot encoding
    
- 모델 선택 및 하이퍼파라미터 조정
- 모델 학습
- 모델 평가

In [ ]:
# get_dummies 함수 이용해서 원- 핫 인코딩 진행시킴
train = pd.get_dummies(train)

In [ ]:
train

In [ ]:
test

In [ ]:
# ------------오류해결 ----------------
test = pd.get_dummies(test)

In [ ]:
test.shape
#원핫인코딩을하면서 트레인에는 없던데이터가 테스트에는 있어서 범주가 하나 더생긴거임.

In [ ]:
 # train, test 컬럼명 차집합 연산
    # train에는 test엔느 없는 범주 데이터가 하나 더 있었음 ! ( 그 데이터를 확인하기 위해서)
set(train.columns) - set(test.columns)

In [ ]:
set(test.columns) - set(train.columns)

In [ ]:
# train데이터와 test데이터의 컬럼수를 맞춰주기 위해 (train에는 있고 test에는 없는) 
# cabin_T= 0으로 놓으면 값이 없다는뜻이니 이렇게 해서 맞춰주기
test['Cabin_T'] = 0

In [ ]:
test.shape

In [ ]:
# 문제, 정답 나누기
X_train = train.drop('Survived', axis =1)
y_train = train['Survived']

In [ ]:
X_test = test
# y_test-> 케글한테

In [ ]:
# 컬럼 확인
X_train.columns

In [ ]:
X_test.columns

In [ ]:
# 컬럼 순서들이 다르니 컬럼 순서들 맞춰줘야함

In [ ]:
# X_test를 X_train의 컬럼 순서대로 가져온걸 다시 X_test에 담고
X_test = X_test[X_train.columns]

In [ ]:
X_train.head(1)
X_test.head(1)

In [ ]:
# test 도 똑같이 인코딩 시켜주려했는데  전체를하려닌까 무슨 시리즈 타입때매 오류가 나서 
# 우회해서 하나씩 인코딩해서 원래 테스트에 붙여넣을거야

In [ ]:
# 원핫 인코딩 시켜줄 컬럼들을 한 변수에 넣어주기
#categorical_features = ['Sex', 'Cabin', 'Embarked', 'Family_Group','Title2']

In [ ]:
#categorical_features

In [ ]:
# 리스트 안에있는 컬럼 이름 하나하나를 꺼내서 원핫인코딩해줄거야
# prefix라는건 접두사 로 어느 컬럼에서 파생된건지 붙여주기 위해서 
#for feature_name in categorical_features:
#    one_hot = pd.get_dummies(test[feature_name], prefix = feature_name)
#    test.drop(feature_name, axis = 1, inplace = True) # 숫자로 인코딩했으닌까 기존에 있던 글자test 데이터를 삭제해주기
#    test = pd.concat([test, one_hot],axis = 1) # 숫자랑 기존데이터 합쳐주기